In [1]:
import io
import requests
import torch
from PIL import Image
from torchvision import models, transforms
from torch.autograd import Variable
from utils.timer import Timer

In [5]:
dtype = torch.cuda.FloatTensor
resnet101=models.resnet101(pretrained=True) 
resnet101.cuda()
resnet101.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [6]:
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Resize(256),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
   normalize
])

In [41]:
#12*9 = 108 tiles
#mosiac size (2688,2016)
img_pil = Image.open('/home/nvidia/Desktop/data/mosaik5.jpg')
w,h = img_pil.size

#img_pil


(2688, 2016)

In [8]:
img_tensor = preprocess(img_pil)
img_tensor.unsqueeze_(0)

tensor([[[[ 1.7694,  1.8379,  1.8037,  ..., -1.1075, -1.1589, -1.1075],
          [ 1.6324,  1.8893,  1.9407,  ..., -1.1247, -1.1418, -1.0733],
          [ 1.6153,  1.9578,  1.9749,  ..., -1.3815, -1.2617, -1.1932],
          ...,
          [ 1.9749,  2.0777,  2.1633,  ...,  0.2624, -0.0116,  0.3994],
          [ 1.8037,  2.0434,  2.1633,  ...,  0.1426, -0.1486,  0.2453],
          [ 1.4440,  1.6838,  1.7694,  ...,  0.1083, -0.1657,  0.1597]],

         [[ 1.8859,  1.9209,  1.8859,  ..., -0.6352, -0.6352, -0.9678],
          [ 1.7108,  1.9734,  2.0609,  ..., -1.0028, -0.7227, -0.9503],
          [ 1.7108,  2.0959,  2.1310,  ..., -1.2829, -1.1954, -1.1779],
          ...,
          [ 2.2010,  2.2885,  2.3410,  ...,  0.3803, -0.1625,  0.2752],
          [ 2.0259,  2.2535,  2.3585,  ...,  0.2227, -0.3200,  0.0476],
          [ 1.6408,  1.8683,  1.9734,  ...,  0.1702, -0.3375, -0.0749]],

         [[ 1.9428,  1.9603,  1.9428,  ..., -0.0964, -0.0790, -0.6541],
          [ 1.7685,  2.0300,  

In [36]:
img_variable = Variable(img_tensor)
timer = Timer()
timer.tic()
out = resnet101(img_variable.cuda())
timer.toc()

0.08867001533508301

In [17]:
LABELS_URL = 'https://s3.amazonaws.com/outcome-blog/imagenet/labels.json'
labels = {int(key):value for (key, value)
          in requests.get(LABELS_URL).json().items()}

In [12]:
labels

{0: u'tench, Tinca tinca',
 1: u'goldfish, Carassius auratus',
 2: u'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 3: u'tiger shark, Galeocerdo cuvieri',
 4: u'hammerhead, hammerhead shark',
 5: u'electric ray, crampfish, numbfish, torpedo',
 6: u'stingray',
 7: u'cock',
 8: u'hen',
 9: u'ostrich, Struthio camelus',
 10: u'brambling, Fringilla montifringilla',
 11: u'goldfinch, Carduelis carduelis',
 12: u'house finch, linnet, Carpodacus mexicanus',
 13: u'junco, snowbird',
 14: u'indigo bunting, indigo finch, indigo bird, Passerina cyanea',
 15: u'robin, American robin, Turdus migratorius',
 16: u'bulbul',
 17: u'jay',
 18: u'magpie',
 19: u'chickadee',
 20: u'water ouzel, dipper',
 21: u'kite',
 22: u'bald eagle, American eagle, Haliaeetus leucocephalus',
 23: u'vulture',
 24: u'great grey owl, great gray owl, Strix nebulosa',
 25: u'European fire salamander, Salamandra salamandra',
 26: u'common newt, Triturus vulgaris',
 27: u'eft',
 28: u's

In [37]:
print(labels[out.data.numpy().argmax()])

TypeError: can't convert CUDA tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [18]:
print("Time for Image classification {:.3f}s ".format(timer.total_time()))

Time for Image classification 0.080s 
